In [ ]:
import numpy as np
import lumicks.pylake as lk
import matplotlib.pyplot as plt
import h5py
from sklearn.linear_model import LinearRegression

In [ ]:
d = h5py.File('/home/kris/proj/.data/tweez/yhsp2.h5', 'r')
signal = d['Trap position']['1X']
first = np.argwhere(np.asarray(signal) < 2)[0][0]
signal = signal[first:]
distance = d['Distance']['Piezo Distance'][first:]
distance = distance - np.amin(distance)
force = d['Force HF']['Force 1x'][first:]

toseconds = 1e-9
duration = (d['Force LF']['Force 1x'][:][-1][0] \
            - d['Force LF']['Force 1x'][:][0][0]) * toseconds
frequency = len(force) / duration
print(duration, frequency)

In [ ]:
plt.plot(signal[::100000])

In [ ]:
np.asarray(signal) < 2
np.argwhere(np.asarray(signal) < 2)

In [ ]:
%matplotlib inline
from util import find_pulls
pulls = find_pulls(signal)
print(len(pulls))

In [ ]:
plt.plot(signal[::100000])
for (start, stop) in pulls:
    plt.axvline(start // 100000, c='tab:green')
    plt.axvline(stop // 100000, c='tab:orange')
    #plt.xlim((50,150))

In [ ]:
from scipy.stats import hmean
target = 5000
pullens = [pull[1] - pull[0] for pull in pulls]
pull_points = int(hmean(pullens))
# pull_points = pulls[0][1] - pulls[0][0]
kernel_size = pull_points // 5000
kernel = np.ones(kernel_size) / kernel_size
smooth_force = np.convolve(force, kernel, mode='same')

In [ ]:
curves = {}
for index, pull in enumerate(pulls):
    identifier = 'curve_' + str(index + 1)
    if not index:
        identifier = identifier + '_0ms'
    else:
        identifier = identifier + '_' + str(int((pull[0] - pulls[index - 1][1]) / frequency * 1000)) + 'ms'
    curves[identifier] = {'force': smooth_force[pull[0]:pull[1]][::kernel_size],
                         'dist': distance[pull[0]:pull[1]][::kernel_size]}
print(curves.keys())

In [ ]:
from util import write_csv

write_csv('/home/kris/proj/.data/tweez/yhsp2.csv', curves)

In [ ]:
for index, curve in enumerate(curves.values()):
    if index < 10:
        continue
    elif index > 20:
        break
    fix,ax1=plt.subplots()
    ax1.plot(curve['force'], c='tab:blue', alpha=0.8)
    ax2 = ax1.twinx()
    ax2.plot(curve['dist'], c='tab:orange', alpha=0.8)

In [ ]:
plt.plot(smooth_force)

In [ ]:
print(bars[0], bars[1])

In [ ]:
import csv
with open('/home/kris/proj/.data/tweez/split_curves.csv', 'w') as f:
    wr = csv.writer(f)
    for curve_id, curve in curves.items():
        wr.writerow([curve_id, *[dist for dist in curve['dist']]])
        wr.writerow([curve_id, *[force for force in curve['force']]])

In [ ]:
for curve in curves.values():
    plt.figure()
    bins = plt.hist(curve['force'], bins=50)
    plt.axhline(np.mean(bins[0]), c='tab:orange')
    plt.axhline(np.mean(bins[0]) + np.std(bins[0]), c='tab:orange', alpha=0.5)
    print(np.mean(bins[0]), np.std(bins[0]))
    plt.figure()
    plt.plot(np.diff(curve['force']))

In [ ]:
def simple_slope(f, index, size):
    return (f[min(len(f) - 1, index + size)] - f[max(0, index - size)]) / (2 * size)

In [ ]:
for curve in curves.values():
    slopes = []
    for index, force in enumerate(curve['force']):
        slopes.append(simple_slope(curve['force'], index, 50))
    plt.plot(slopes)
    #plt.plot(curve['force'])
plt.axhline(0)

In [ ]:
single = curves['curve_1_0ms']
slopes = []
for index, force in enumerate(single['force']):
    slopes.append(simple_slope(single['force'], index, 50))
plt.plot(slopes)

subzeros = []
for index, slope in enumerate(slopes[1000:-1000]):
    if slope < 0:
        subzeros.append(index)
for index, sub in enumerate(reversed(subzeros)):
    if subzeros[-1] == subzeros[-2] + 1:
        subzeros.pop(-1)
    else:
        break
    #print(sub, index)
print(len(subzeros))
plt.axvline(subzeros[0] + 1000)
plt.axvline(subzeros[-1] + 1000)
plt.figure()
plt.plot(single['force'])
plt.axvline(subzeros[0] + 1000)
plt.axvline(subzeros[-1] + 1000)
